# Student Loan Interest Calculator + Visualization
Runs various studies on loans to determine the optimal means for paying off the loan.

Emily's Loans
- Loan Manager: FedLoanServicing
- Loan 1: 6507.64, 6.410% APR
- Loan 2: 21263.06, 5.410% APR
- Loan 3: 6994.65, 7.900% APR
- Loan 4: 23562.73, 6.800% APR

The following events will trigger interest recapitalization:
1. When the loan enters repayment.
2. When a deferment ends.
3. When forbearance ends.
4. When the loan defaults.
5. A change in repayment plan.
6. Loan consolidation. 

Main source of information: https://studentaid.ed.gov

Definition of Terms
1. <b>Enters repayment</b>: you leave school and now must pay your loans back.
2. <b>Deferment Ends</b>: A deferment occurs when you stop making payments on your student loans. It has special rules about when you have to pay the interest on your loans during a deferment.
 - During a deferment, you generally don't have to pay for interest if you have a subsidized direct, federal stafford, direct consolodated, or FFEL consolidated loans. This is because whoever subsidizes your loan is paying the interest as it occurs.
 - You <b>must</b> pay interest on all subsidized loans.
 - Deferment is basically the default status of a student loan while you are in school, and generally, if you don't pay the accrued interest on unsubsidized loans, this unpaid interest gets added to your principal loan balance, which then increases the total interest payment, because now you effectively have a larger loan.
3. <b>Forbearance Ends</b>:  
 - Forbearance is essentially the same thing as a deferrment in that you tell your loan servicer that you cannot make payments against your principal balance. The difference is that you are always responsible for paying the interest during a forbearance. After forbearance ends, unpaid interest is recapitalized.
 - Forbearance and Deferment both free you from making loan payments against your principal, but you're just burning money as interest.
4. <b>Default</b>: Student loan default when you fail to make loan payments as scheduled.
 - When you default on your loan, legal action can be taken to force you, or the person who cosigned your loan (a guarantor), or both to pay the balance of the loan. This can happen in a broad variety of very unpleasant ways, such as wage garnishing, repossession of property, etc.
 - Usually, prior to going into default, you enter a period of <b>delinquency</b> which means you haven't made the loan payments you agreed to when you signed your promisary note upon applying for the loan in the first place. Being delinquent has bad consequences too, it effects your credit score, and can prevent you from entering into financial arrangements with entities, such as getting other loans, getting approval to rent an apartment, getting homeowners' insurance, or even signing up for utilities.
 - If you pay your loan monthly, default occurs after missing a payment for 270 days.
 - There is no guaranteed way to exit default, you have to contact your loan servicer or collections agency.
 - <b> There are huge consequences to going into default. Its really bad. </b>
 	- You must pay the entire balance of your loans immediately (obviously impossible), you lose elgibility for deferment or forbearance, you lose elgiability for all federal student loans, your loan may go to collections, your credit score tanks, your federal tax return may be given to the debt collectors, your total debt will balloon with fees and unpaid interest, your wages can be garnished, you may be sued and blocked from purchasing or selling certain assets (such as a house), federal employees may sacrifice 15% of pay, and you must spend years to re-establish your credit rating.
  
  

In [12]:
%matplotlib notebook
import matplotlib.pyplot as plt
import numpy as np
import collections

class LoanSolver(object):
    ''' Loans are added and stored as dictionaries in self.loans list'''
    def __init__(self):
        self.loans = {}
        # Stores all loans which have been paid off at their initial state.
        self.paid_loans = {}
        
        # Convert time into years by supplying the appropriate unit
        self._allowed_time_units = {
            'hour':lambda x:x/8760.,
            'day':lambda x:x/365.,
            'minute':lambda x:x/525600.,
            'second':lambda x:x/31536000.,
            'year':lambda x:x,
            'month':lambda x:x/12.,
        }
        
        # Factory methods for supported loan types - different 
        # loans may have different attributes, so each loan 
        # type gets its own generator.
        self._loan_types = {
            'StudentLoan':collections.namedtuple('StudentLoan', 'name rate type interest principal' )
        }
        
    def add_loan(self, loan_name, loan_type, interest_rate, principal_balance, interest_balance=0.):
        '''Adds a loan to the loan set. Interest rate is assumed to be APR 
        
        :param: loan_name (str) - The name for the loan, must be unique
        :param: loan_type (str) - The type of loan, defines the rules for
            interest recapitalization and other specific loan information
        :param: interest_rate (float) - yearly fractional interest rate 
            (ex 0.5 = 50%) for a whole year
           
        Math assumes time to be in years, therefore all times are 
        expressed internally as fractions of years so as to use the familiar
        compounding interest formula:
        
            F(t) = Principal * exp( rate * time_in_years ) 
            
        '''
        # generate new instance of StudentLoan (add more loan types as needed)
        if loan_type not in self._loan_types:
            raise ValueError('Loan type {} is not yet supported. Available types are {}'.format(loan_type, repr(self._loan_types.keys())))
        if loan_name in self.loans:
            raise ValueError('Loan with name {} already exists'.format(loan_name))
        loan = self._loan_types[loan_type](
            name=loan_name, 
            rate=interest_rate, 
            type=loan_type,
            principal=principal_balance,
            interest=interest_balance,
        )
        print(loan)
        self.loans[loan_name]=loan
        if loan not in self.paid_loans:
            self.paid_loans[loan_name] = {'loan':loan, 'paid_off':False} # True when paid
        
    def recapitalize(self, loan_type=None, loan_name=None):
        '''Recapitalizes the interest in all loans in self.loans. If 
        loan_name is specified, all loans mathing the name will be 
        recapitalized. If loan_type is specified, all loans matching 
        that type will be recapitalized. If loan_name and loan_type 
        are specified, only loans matching the name and type will be 
        recapitalized.
        
        :param: loan_name (str) recapitalize only loans matching this name
        :param: loan_type (str) recapitalize only loans matching this name
        
        Since loan_name is already enough to uniquely identify a loan, 
        the loan_type is ignored in cases where loan_type is specified.
        
        If loan_name, loan_type are not set, then all loans 
        will be recapitalized.
        '''
        
        for loan in self.loans.values():
            if loan_type is not None and loan_name is None:
                if loan_type == loan.type:
                    loan.principal += loan.interest
                    loan.interest = 0
            if loan_name is not None:
                if loan_name == loan.name:
                    loan.principal += loan.interest
                    loan.interest = 0
            else:
                loan.principal += loan.interest
                loan.interest = 0
                
    def add_interest(self, amount_of_time, time_unit='years'):
        unit = time_unit.lower()
        if unit not in self._allowed_time_units:
            raise ValueError("Allowed time units are: {}".repr(self._allowed_time_units.keys()))
        delta_t = self._allowed_time_units[unit](amount_of_time)
        
        for loan in self.loans.values():
            loan.interest = loan.principal * np.exp(loan.rate * delta_t) - loan.principal
    
    def pay_loan(self, loan_name, amount):
        ''' pay amount to loan_name. Payment applies first to interest, and then to principal
        Returns either 0 or unspent money'''
        
        if payment <= self.loans[loan_name].interest:
            self.loans[loan_name] -= payment
            return 0
        else:
            payment -= self.loans[loan_name].interest
            self.loans[loan_name].interest = 0.
        if payment >= self.loans[loan_name].balance:
            payment -= self.loans[loan_name].balance
            self.loans[loan_name].balance = 0.
            self.paid_loans[loan_name]['paid_off'] = True
            del self.loans[loan_name]
            return payment
        else:
            self.loans[loan_name].balance -= payment
            return 0
    def largest_interest_rate(self):
        return sorted(self.loans.values(), key=lambda x:x[1].interest)[-1][1].name
    
    def smallest_interest_rate(self):
        return sorted(self.loans.values(), key=lambda x:x[1].interest)[0][1].name
    
    def largest_principal(self):
        return sorted(self.loans.values(), key=lambda x:x[1].principal)[0][1].name
    
    def smallest_principal(self):
        return sorted(self.loans.values(), key=lambda x:x[1].principal)[0][1].name
        
        
    

In [13]:
solver = LoanSolver()
solver.add_loan(loan_name='test',loan_type='StudentLoan',interest_rate=0.054,principal_balance='60000')

StudentLoan(name='test', rate=0.054, type='StudentLoan', interest=0.0, principal='60000')
